In [14]:
!pip install oursin==0.6.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 8.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.9/407.9 kB 10.2 MB/s eta 0:00:0000:01
INFO: pip is looking at multiple versions of vbl-aquarium to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of python-socketio[client] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.9 MB/s eta 0:00:00
  Using cached python_socketio-5.9.0-py3-none-any.whl (57 kB)
  Using cached python_socketio-5.8.0-py3-none-any.whl (56 kB)
INFO: pip is looking at multiple versions of vbl-aquarium to de

In [1]:
import pandas as pd
import numpy as np
import oursin as urchin

from iblatlas.atlas import AllenAtlas
atlas = AllenAtlas(25)

from matplotlib import cm

turbo_cmap = cm.get_cmap('turbo')

/var/folders/dj/b6xhnrzd69s_5s9jyt2r8cfr0000gn/T/ipykernel_26364/2727036022.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  turbo_cmap = cm.get_cmap('turbo')


In [2]:
data = pd.read_csv('figure7_dataframe_filt.csv')

In [3]:
data

,Unnamed: 0.1,cluster_ids,cluster_idx,avg_fr,avg_fr_base,avg_fr_trial,avg_fr_rxn_time,avg_fr_post_stim,avg_fr_pre_move,avg_fr_pre_moveL,...,high_noise,high_lfp,low_yield,missed_target,low_trials,include,artifacts,lab_include,permute_include,decode_no_qc_include
0,0,374,374,10.609782,11.334405,11.655949,9.806691,10.096463,10.353698,10.720721,...,False,False,False,False,False,True,False,True,True,True
1,1,375,375,6.506971,5.643087,6.615756,6.149661,7.266881,7.159700,5.585586,...,False,False,False,False,False,True,False,True,True,True
2,2,380,380,10.803333,7.845659,11.535370,8.673311,8.295820,8.981779,8.963964,...,False,False,False,False,False,True,False,True,True,True
3,3,393,393,8.535078,5.819936,9.397106,6.158469,6.398714,6.795284,6.396396,...,False,False,False,False,False,True,False,True,True,True
4,4,397,397,3.833393,5.000000,4.180064,3.704674,3.601286,4.222937,4.189189,...,False,False,False,False,False,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4847,5307,1304,1304,1.421378,1.169451,0.763723,0.881807,1.026253,0.731901,0.630182,...,False,False,False,False,False,True,False,True,True,True
4848,5308,1307,1307,9.741949,8.806683,8.854415,8.917067,8.735084,8.766905,10.547264,...,False,False,False,False,False,True,False,True,True,True
4849,5309,1355,1355,9.147455,5.143198,7.368735,8.735898,8.568019,7.446301,6.135987,...,False,False,False,False,False,True,False,True,True,True
4850,5310,1523,1523,1.761905,1.467780,0.924821,1.347112,1.288783,0.843278,0.796020,...,False,False,False,False,False,True,False,True,True,True


In [55]:
FR_thresh = 12.3
bregma = atlas.xyz2ccf([0,0,0])

neuronCount = 0
neuronNames = []
neuronLocs = []
neuronColors = []
neuronSizes = []
neuronShapes = []

for (i, row) in data.iterrows():
    ccf = atlas.xyz2ccf([row.x + np.random.normal()/200000, row.y, row.z + np.random.normal()/200000])
    
    if atlas.get_labels([row.x, row.y, row.z]) == 218:
        neuronLocs.append(ccf.tolist())
        neuronSizes.append([0.03, 0.03, 0.03] if row.avg_fr > FR_thresh else [0.015, 0.015, 0.015])
        neuronShapes.append('sphere')
        
        # color
        color = turbo_cmap(row.avg_fr/20)
        neuronColors.append(color[:3])

In [5]:
urchin.setup()

(URN) connected to server
Login sent with ID: 1fa93c52, copy this ID into the renderer to connect.


In [6]:
urchin.ccf25.load()

In [40]:
urchin.ccf25.grey.set_visibility(True, urchin.utils.Side.FULL)
urchin.ccf25.grey.set_material('transparent-unlit', urchin.utils.Side.FULL)
urchin.ccf25.grey.set_alpha(0.1, urchin.utils.Side.FULL)

urchin.ccf25.LP.set_visibility(True, urchin.utils.Side.LEFT)
urchin.ccf25.LP.set_material('transparent-lit', urchin.utils.Side.LEFT)
urchin.ccf25.LP.set_alpha(0.25, urchin.utils.Side.LEFT)

In [39]:
urchin.camera.main.set_mode('perspective')
urchin.camera.main.set_rotation([22.5,22.5,225])

In [56]:
neurons = urchin.meshes.create(len(neuronLocs))
urchin.meshes.set_positions(neurons, neuronLocs)
urchin.meshes.set_scales(neurons, neuronSizes)
urchin.meshes.set_colors(neurons, neuronColors)

In [57]:
test = urchin.meshes.Mesh()

In [63]:
test.set_scale([1,1,1])
test.set_color([1,0,0])